# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( '/Users/sowmyamoka/Satish_Python/data_wrangling_xml/data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print( capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( '/Users/sowmyamoka/Satish_Python/data_wrangling_xml/data/mondial_database.xml' )

In [8]:
root2=document.getroot()

In [54]:
import pandas as pd
list_country=[1,2]
list_inmr=[3,4]
for country in root2.findall('country'):
    
   if((country.find('infant_mortality')==None)|(country.find('name')==None)) :  
       continue
   else :
        country_name = country.find('name').text
        inmrt = country.find('infant_mortality').text
        list_country.append(country_name)
        list_inmr.append(inmrt)
dic_inmr={'country':list_country,'inmr':list_inmr}        
df=pd.DataFrame(dic_inmr)
df['inmr_numeric']=pd.to_numeric(df.inmr)
del df['inmr']
df.sort_values(by='inmr_numeric',ascending=True).head(10)



,country,inmr_numeric
38,Monaco,1.81
92,Japan,2.13
36,Norway,2.48
111,Bermuda,2.48
100,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
74,Hong Kong,2.73
0,1,3.00
75,Macao,3.13


In [177]:
import pandas as pd
list_city_name=[]
list_year=[]
list_population=[]
for city in root2.iter('city'):
    if((city.find('name')==None)|(city.find('population')==None)):  
        continue        
    else :
        city_name = city.find('name').text
        for pop in city.findall('population'):
             year= pop.get('year')
             population=pop.text             
             list_city_name.append(city_name)
             list_year.append(year)
             list_population.append(population)   
    
dic={'city':list_city_name,'year':list_year,'population':list_population}
df=pd.DataFrame(dic)
df['population']=pd.to_numeric(df['population'])
df['year']=pd.to_numeric(df['year'])
df.head(10)
df=df.ix[df.groupby('city')['year'].idxmax]
df.sort_values('population',ascending=False).head(10)

,city,population,year
3750,Shanghai,22315474,2010
2607,Istanbul,13710512,2012
4303,Mumbai,12442373,2011
1546,Moskva,11979529,2013
3746,Beijing,11716620,2010
8208,São Paulo,11152344,2010
3754,Tianjin,11090314,2010
3364,Guangzhou,11071424,2010
4399,Delhi,11034555,2011
3371,Shenzhen,10358381,2010


In [197]:
import pandas as pd

list_year=[]
list_population=[]
list_ethnicity=[]
list_ethnicity_per=[]
list_country=[]
for country in root2.findall('country'):
    for pop in country.findall('population'):
        
        for ethnic in country.findall('ethnicgroup'):
            year= pop.get('year')
            name=country.find('name').text
            population=pop.text
            ethnicity=ethnic.text
            ethnicity_per=ethnic.get('percentage')
            list_year.append(year)
            list_population.append(population)
            list_ethnicity.append(ethnicity)
            list_ethnicity_per.append(ethnicity_per)
            list_country.append(name)
            
dic_enthnicity={'country':list_country,'ethnicity':list_ethnicity,'ethnicity_per':list_ethnicity_per}           
dic_country={'country':list_country,'year':list_year, 'population':list_population }     
df1=pd.DataFrame(dic_enthnicity)
df2=pd.DataFrame(dic_country)
df1['ethnicity_per']=pd.to_numeric(df['ethnicity_per'])
df2['population']=pd.to_numeric(df2['population'])
df2['year']=pd.to_numeric(df2['year'])
df1=df1.drop_duplicates()
df2=df2.drop_duplicates()
df2=df2.ix[df2.groupby('country')['year'].idxmax]
df3=df1.merge(df2)
df3=df3[['ethnicity','population']]
df3.groupby('ethnicity').population.sum().sort_values(ascending=False).head(10)




ethnicity
Han Chinese    1360720000
Mongol         1213609662
Dravidian      1210854977
Indo-Aryan     1210854977
European       1157295639
African         975352746
Amerindian      588752467
Malay           377500275
Asian           374650120
Russian         322438406
Name: population, dtype: int64

In [155]:
river_name_list=[]
river_length_list=[]
river_country_list=[]
lake_name_list=[]
lake_area_list=[]
lake_country_list=[]
airport_name_list=[]
airport_elevation_list=[]
airport_country_list=[]
import pandas as pd
for river in root2.findall('river'):
      if((river.find('name')==None)|(river.find('length')==None)|(river.get('country')==None)):  
        continue  
    
      else:
        river.find('name')
        river.find('length')
        river.get('country')
        river_name= river.find('name').text
        river_length=river.find('length').text 
        river_country=river.get('country')
        river_length_list.append(river_length)  
        river_name_list.append(river_name)
        river_country_list.append(river_country)
for lake in root2.findall('lake'):
      if((lake.find('name')==None)|(lake.find('area')==None)|(lake.get('country')==None)):  
        continue  
    
      else:
        lake.find('name')
        lake.find('area')
        lake.get('country')
        lake_name= lake.find('name').text
        lake_area=lake.find('area').text 
        lake_country=lake.get('country')
        lake_area_list.append(lake_area)  
        lake_name_list.append(lake_name)
        lake_country_list.append(lake_country)
        
for airport in root2.findall('airport'):
      if((airport.find('name')==None)|(airport.find('elevation')==None)|(airport.get('country')==None)):  
        continue  
    
      else:
        airport.find('name')
        airport.find('area')
        airport.get('country')
        airport_name= airport.find('name').text
        airport_elevation=airport.find('elevation').text 
        airport_country=airport.get('country')
        airport_elevation_list.append(airport_elevation)  
        airport_name_list.append(airport_name)
        airport_country_list.append(airport_country)
        
data_river=pd.DataFrame({'name':river_name_list,'parameter':river_length_list,'country':river_country_list})
data_river['group']='river'
data_lake=pd.DataFrame({'name':lake_name_list,'parameter':lake_area_list,'country':lake_country_list})
data_lake['group']='lake'
data_airport=pd.DataFrame({'name':airport_name_list,'parameter':airport_elevation_list,'country':airport_country_list})
data_airport['group']='airport'
data_aggregate=pd.concat([data_river,data_lake,data_airport],ignore_index=True)
data_aggregate['parameter']=pd.to_numeric(data_aggregate['parameter'])
data_aggregate.groupby('group')['parameter'].idxmax
data_aggregate=data_aggregate.ix[data_aggregate.groupby('group')['parameter'].idxmax]
data_aggregate

,country,name,parameter,group
452,BOL,El Alto Intl,4063.0,airport
287,R AZ KAZ IR TM,Caspian Sea,386400.0,lake
174,CO BR PE,Amazonas,6448.0,river
